In [ ]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime
from requests.exceptions import HTTPError
from datetime import datetime


In [ ]:
def is_minute_multiple_of_five():
    """
    Check if the current minute is a multiple of 5.
    
    :return: True if the current minute is a multiple of 5, otherwise False.
    """
    current_minute = datetime.now().minute
    return current_minute % 5 == 0

In [ ]:
def get_market_orders_jita(page=1, station_id=60003760, base_url="https://esi.evetech.net/latest"):
    """
    Fetch market orders for Jita from ESI API.
    
    :param page: Page number for pagination.
    :return: JSON response containing market orders.
    """
    url = f"{base_url}/markets/10000002/orders/"
    params = {
        "order_type": "sell",
        "page": page,
        "structure_id": station_id
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # This will raise an HTTPError for bad responses
    
    return response.json()

In [ ]:
def download_all_orders(station_id=60003760, base_url="https://esi.evetech.net/latest"):
    """
    Download all sell orders from Jita.
    
    :return: A list containing all market orders.
    """
    all_orders = []
    page = 1
    print("Downloading market orders.")
    while True:
        try:
            orders = get_market_orders_jita(page, station_id, base_url)
            if not orders:
                break
            all_orders.extend(orders)
            # print(f"Fetched page {page}, orders: {len(orders)}")
            page += 1
            time.sleep(1)  # Limit request rate
        except HTTPError as e:
            if e.response.status_code == 404:
                # print(f"Page {page} not found. Ending download.")
                break
            else:
                raise  # Re-raise the exception if it's not a 404 error
    
    # Convert the list of orders to a DataFrame
    all_orders = pd.DataFrame(all_orders)
    print("Download complete.")
    return all_orders



In [ ]:
def snipe(orders, type_id, station_id, threshold = 0.5):
    # get all orders for type_id
    # orders = data[data['type_id'] == type_id]
    # get the lowest sell order
    lowest_sell = orders[orders['is_buy_order'] == False]['price'].min()

    # get the highest buy order
    highest_buy = orders[orders['is_buy_order'] == True]['price'].max()
    
    # get second lowest sell order:
    try:
        second_lowest_sell = orders[orders['is_buy_order'] == False]['price'].sort_values().iloc[1]
    except(IndexError):
        second_lowest_sell = lowest_sell
        margin = 0
        
    spread = abs(highest_buy - lowest_sell)
    difference_to_next_sell = abs(second_lowest_sell - lowest_sell)

    # check if the spread between buy/sell is smaller than the spread between the lowest sell and the second lowest sell
    # if TRUE then we have a snipe - we can buy at the lowest sell price and relist for an easy profit (in theory)
    # This should be more dynamic than simply checking for an exact match between the buy/sell spread 
    if spread < (difference_to_next_sell * threshold) or lowest_sell == highest_buy:


        lowest_sell_volume = orders[orders['is_buy_order'] == False]['volume_remain'].iloc[-1]
        
        
        
        
        margin = second_lowest_sell - lowest_sell
        # volume = orders[orders['is_buy_order'] == False]['volume'].sum()
        
        print(f"Type ID {type_id} is a snipe at station: {station_id}.") 
        print(f"Lowest sell: {lowest_sell} Highest buy: {highest_buy} volume: {lowest_sell_volume}") 
        print(f"Second sell: {second_lowest_sell} ~ Margin: {margin} ISK")
        print(f"Profit: {margin*lowest_sell_volume} ISK")
        return True
    else:
        return False


In [ ]:
def filter_data(data, station_id, threshold):

    # data = data[data['location_id'] == station_id]
    station_ids = data['location_id'].unique()
    type_ids = data['type_id'].unique()
    # for station_id in station_ids:
    station_data = data[data['location_id'] == station_id]
    snipeFound = False
    for type_id in type_ids:
        orders = station_data[station_data['type_id'] == type_id]
        snipeFound = snipe(orders, type_id, station_id, threshold=threshold)
    if snipeFound == False:
            print("          No snipes found. Back to sleep.")
            print("==========================================")

In [ ]:
import time
from datetime import datetime

def check_five_minute():
    """
    Check if the current minute is a multiple of 5.
    
    :return: True if the current minute is a multiple of 5, otherwise False.
    """
    current_minute = datetime.now().minute
    return current_minute % 5 == 0

def snipeScan(station_id, base_url, threshold=0.5):
    """
    Run the is_minute_multiple_of_five function every 10 seconds,
    but only once per minute when the minute is a multiple of 5.
    """
    last_checked_minute = None
    
    print("==========================================")
    print("             SnipeScan running")
    print("==========================================")

    print(f"Running Snipe Scan at {datetime.now().minute}.")
    last_checked_minute = datetime.now().minute
    # Run the snipe scan
    raw_data = download_all_orders(station_id=station_id, base_url=base_url)
    filter_data(raw_data, station_id, threshold)

    while True:
        current_minute = datetime.now().minute
        
        if current_minute != last_checked_minute:
            if check_five_minute():
                print(f"Running Snipe Scan at {datetime.now()}.")
                last_checked_minute = current_minute
                # Run the snipe scan
                raw_data = download_all_orders(station_id=station_id, base_url=base_url)
                filter_data(raw_data, station_id, threshold)
                
        # Wait for 10 seconds before checking again
        time.sleep(20)




In [ ]:
# test_data = pd.read_csv('data/2024_08_11/market-orders-2024-08-11_01-15-06.v3.csv')

# station_ids = {
#     'Amarr': 60008494,
#     'Jita': 60003760,
#     'Dodixie': 60011866,
#     'Hek': 60005686,
#     'Rens': 60004588
# }

# for station_id in station_ids.values():
#     filter_data(test_data, station_id, threshold=0.5)

In [ ]:

# Constants
snipe_station_id = 60008494  # Jita IV - Moon 4 - Caldari Navy Assembly Plant
base_url = "https://esi.evetech.net/latest"

# Start the loop
snipeScan(station_id=snipe_station_id, base_url=base_url, threshold=0.8)